In [138]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions
using OrthoMatchingPursuit
using BenchmarkTools

In [164]:
Nx = 40
Ne = 500

X = randn(Nx, Ne) .* randn(Nx, Ne)

λ = 0.0
δ = 1e-8
γ = 2.0

2.0

In [52]:
@time Cgreedy = greedyfit(Nx, 2, X, 3, λ, δ, γ)

LoadError: [91mBoundsError: attempt to access 5-element Array{Float64,1} at index [2:501][39m

In [165]:
p = 3
order = p*ones(Int64, Nx)
order[1] = p
order[end] = p

# order = [p; p; -1; p; p]

C = SparseRadialMapComponent(Nx, order)
@time center_std!(C, sort(X; dims = 2); γ = γ)

  0.001304 seconds (111 allocations: 635.734 KiB)


In [166]:
@time ψ_off, ψ_diag, dψ_diag = compute_weights(C, X)

  0.001375 seconds (20.01 k allocations: 1.549 MiB)


([-0.07695495347435113 0.5371305976468275 … 0.5348353415112977 0.40564703355858556; -0.07843913267527085 0.5375737361029362 … 0.21647071327566844 0.3778485841664432; … ; 3.5272393294695923 8.518160684460277e-8 … 0.46538164404971155 0.301342342566144; -0.09929831942511107 0.543574330101049 … 0.5205995856227194 0.59789931475629], [-0.02305457857860134 0.934855732618143 … 0.8039505784502463 0.4416742073331542; -0.09686341438360355 0.6918074852983874 … 0.48017314058118177 0.19054426959353898; … ; -0.2515344780458789 0.3575064180131047 … 0.20476333499066535 0.07682429167702692; -0.1343915358319392 0.5898666675570672 … 0.38411186032521183 0.14586394335584407], [0.060995829535610224 0.2097934742193318 … 0.40883112115431486 0.5281327657337088; 0.1986337601548132 0.5812405062517979 … 0.5889126085624522 0.29696560464754285; … ; 0.4022798806367571 0.616460664657699 … 0.41965232972656025 0.14689381320288247; 0.2555428516089264 0.6421647221871366 … 0.5645844649599199 0.24306503593460038])

In [167]:
@time x_opt = optimize(C, X, λ, δ)
modify_a!(C, x_opt)

  0.018458 seconds (26.41 k allocations: 6.956 MiB, 65.91% gc time)


In [168]:
# Compute centers and widths
center_std!(C, sort(X; dims = 2); γ = γ)

### Evaluate the different basis

# Create weights
ψ_off, ψ_diag, dψ_diag = compute_weights(C, X);

In [169]:
n_off = size(ψ_off,2)
n_diag = size(ψ_diag, 2)
ψ_aug = vcat(ψ_off, Matrix(0.1*I, n_off, n_off));

In [170]:
@time F = qr(ψ_aug);

  0.002951 seconds (7 allocations: 887.516 KiB)


In [171]:
P1 = Matrix(1.0*I, Ne, Ne+n_off)
P2 = Matrix(1.0I, Ne + n_off, n_off);

In [172]:
@time Q1 = Matrix(F.Q)[1:Ne,:];

  0.001363 seconds (14 allocations: 2.343 MiB)


In [173]:
@time P1*F.Q*P2;

  0.003399 seconds (8 allocations: 3.693 MiB)


In [174]:
@time norm(P1*F.Q*P2-Q1)

  0.003982 seconds (11 allocations: 4.288 MiB)


7.490120976457145e-15

In [175]:
f(ψ, Q, N, nx) = ([([ψ zeros(size(ψ,1), nx)] * Q)[axes(ψ,1), 1:nx] zeros(size(ψ,1), N)] * Q')[axes(ψ_diag,1), 1:N]

f (generic function with 1 method)

In [176]:
size(Q1)

(500, 156)

In [227]:
Q1 = Matrix(F.Q)[1:Ne,:]
result = Q1*(Q1'*ψ_diag)

500×5 Array{Float64,2}:
 -0.228567   0.724844  0.741123  0.63553    0.502047
 -0.0780067  0.678653  0.625237  0.538995   0.379566
 -0.288603   0.474463  0.379082  0.299534   0.201762
 -0.303493   0.559745  0.490935  0.355655  -0.0988953
 -0.248312   0.463409  0.259394  0.283988   0.146939
 -0.321962   0.655918  0.605441  0.530675   0.458575
 -0.299837   0.525227  0.425848  0.385368   0.406331
 -0.315663   0.673176  0.677037  0.525597   0.492572
 -0.154003   0.50254   0.341729  0.362911   0.237803
 -0.391999   0.486556  0.369566  0.340983   0.164849
 -0.313052   0.413892  0.3107    0.30044    0.0944667
 -0.309352   0.552295  0.475051  0.402166   0.141449
 -0.316085   0.576557  0.519824  0.447694   0.399158
  ⋮                                        
 -1.59639    0.462156  0.443169  0.433621   0.440629
 -0.382115   0.604193  0.611211  0.484118   0.159471
 -0.55842    0.434862  0.321896  0.350913   0.636623
 -0.253372   0.529607  0.434203  0.441014   0.882692
 -0.111831   0.649595  0.6171

In [178]:
@time P1*F.Q*(P2*P2')*(F.Q'*(P1'*ψ_diag))

  0.009008 seconds (22 allocations: 8.959 MiB)


500×5 Array{Float64,2}:
 -0.228567   0.724844  0.741123  0.63553    0.502047
 -0.0780067  0.678653  0.625237  0.538995   0.379566
 -0.288603   0.474463  0.379082  0.299534   0.201762
 -0.303493   0.559745  0.490935  0.355655  -0.0988953
 -0.248312   0.463409  0.259394  0.283988   0.146939
 -0.321962   0.655918  0.605441  0.530675   0.458575
 -0.299837   0.525227  0.425848  0.385368   0.406331
 -0.315663   0.673176  0.677037  0.525597   0.492572
 -0.154003   0.50254   0.341729  0.362911   0.237803
 -0.391999   0.486556  0.369566  0.340983   0.164849
 -0.313052   0.413892  0.3107    0.30044    0.0944667
 -0.309352   0.552295  0.475051  0.402166   0.141449
 -0.316085   0.576557  0.519824  0.447694   0.399158
  ⋮                                        
 -1.59639    0.462156  0.443169  0.433621   0.440629
 -0.382115   0.604193  0.611211  0.484118   0.159471
 -0.55842    0.434862  0.321896  0.350913   0.636623
 -0.253372   0.529607  0.434203  0.441014   0.882692
 -0.111831   0.649595  0.6171

In [212]:
P1

500×656 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [213]:
P1*F.Q*(P2*P2')*(F.Q'*[ψ_diag; zeros(size(ψ_off,2), size(ψ_diag,2))])

500×5 Array{Float64,2}:
 -0.228567   0.724844  0.741123  0.63553    0.502047
 -0.0780067  0.678653  0.625237  0.538995   0.379566
 -0.288603   0.474463  0.379082  0.299534   0.201762
 -0.303493   0.559745  0.490935  0.355655  -0.0988953
 -0.248312   0.463409  0.259394  0.283988   0.146939
 -0.321962   0.655918  0.605441  0.530675   0.458575
 -0.299837   0.525227  0.425848  0.385368   0.406331
 -0.315663   0.673176  0.677037  0.525597   0.492572
 -0.154003   0.50254   0.341729  0.362911   0.237803
 -0.391999   0.486556  0.369566  0.340983   0.164849
 -0.313052   0.413892  0.3107    0.30044    0.0944667
 -0.309352   0.552295  0.475051  0.402166   0.141449
 -0.316085   0.576557  0.519824  0.447694   0.399158
  ⋮                                        
 -1.59639    0.462156  0.443169  0.433621   0.440629
 -0.382115   0.604193  0.611211  0.484118   0.159471
 -0.55842    0.434862  0.321896  0.350913   0.636623
 -0.253372   0.529607  0.434203  0.441014   0.882692
 -0.111831   0.649595  0.6171

In [217]:
axes(ψ_diag,1)

Base.OneTo(500)

In [233]:
norm(fast_mul(ψ_diag, F.Q, Ne, n_off)-result)

2.3588117956006428e-14

In [225]:
@time (F.Q*[(F.Q'*[ψ_diag; zeros(size(ψ_off,2), size(ψ_diag,2))])[axes(ψ_off,2), 1:size(ψ_diag,2)]; zeros(size(ψ_off,1), size(ψ_diag,2))])[axes(ψ_off,1), 1:size(ψ_diag,2)]

  0.000448 seconds (25 allocations: 167.344 KiB)


500×5 Array{Float64,2}:
 -0.228567   0.724844  0.741123  0.63553    0.502047
 -0.0780067  0.678653  0.625237  0.538995   0.379566
 -0.288603   0.474463  0.379082  0.299534   0.201762
 -0.303493   0.559745  0.490935  0.355655  -0.0988953
 -0.248312   0.463409  0.259394  0.283988   0.146939
 -0.321962   0.655918  0.605441  0.530675   0.458575
 -0.299837   0.525227  0.425848  0.385368   0.406331
 -0.315663   0.673176  0.677037  0.525597   0.492572
 -0.154003   0.50254   0.341729  0.362911   0.237803
 -0.391999   0.486556  0.369566  0.340983   0.164849
 -0.313052   0.413892  0.3107    0.30044    0.0944667
 -0.309352   0.552295  0.475051  0.402166   0.141449
 -0.316085   0.576557  0.519824  0.447694   0.399158
  ⋮                                        
 -1.59639    0.462156  0.443169  0.433621   0.440629
 -0.382115   0.604193  0.611211  0.484118   0.159471
 -0.55842    0.434862  0.321896  0.350913   0.636623
 -0.253372   0.529607  0.434203  0.441014   0.882692
 -0.111831   0.649595  0.6171

In [202]:
@time (F.Q'*[ψ_diag; zeros(size(ψ_off,2), size(ψ_diag,2))])[1:size(ψ_off,2), 1:size(ψ_diag,2)]

  0.000316 seconds (12 allocations: 70.312 KiB)


156×5 Array{Float64,2}:
 -0.298088    -0.016012      0.0112515  -0.00316693   0.299024
  5.56465    -11.8034      -10.2621     -8.9304      -5.76346
  0.574319    -1.5218       -1.48219    -1.10915     -0.358545
 -1.11981      3.91647       3.41947     2.96622      2.239
  0.251255    -0.167548     -0.126288   -0.203863    -0.169365
  0.779539    -2.17904      -2.04059    -1.74775     -1.20624
 -0.207295    -1.40107      -1.45695    -1.20246     -1.25118
 -0.856089     0.833358      0.613781    0.504987     0.434846
 -0.115483    -0.140538     -0.18914    -0.105393    -0.44225
  1.05774     -1.06016      -0.971209   -0.855864    -0.219547
  0.569336     0.633846      0.561717    0.403409    -0.589793
 -0.291582     0.846187      0.755927    0.591592    -0.112997
 -0.422389    -0.256639     -0.269124   -0.164191    -0.622013
  ⋮                                                  
  0.0490544   -0.20672      -0.226392   -0.314176    -1.04177
  0.244409     0.202387      0.137055    0.25887

In [152]:
@time begin 
    Q1 = Matrix(F.Q)[1:Ne,:]
    result = Q1*(Q1'*ψ_diag)
end

  0.001429 seconds (18 allocations: 2.368 MiB)


500×5 Array{Float64,2}:
 -0.728446    0.448605    0.37658     0.309789     0.179603
 -0.325852    0.537468    0.428857    0.353472     0.355624
  0.110484    0.754136    0.666182    0.553565     0.454329
 -0.242463    0.656166    0.630315    0.572041     0.657237
 -0.331703    0.31245     0.203483    0.184015    -0.0125278
 -0.174567    0.804569    0.789197    0.619501     0.894336
 -0.286815    0.581218    0.557305    0.385129     0.056356
 -0.209634    0.680492    0.608784    0.572595     0.387254
 -0.168296    0.707425    0.700517    0.541264     0.261889
  0.04345     0.716067    0.648536    0.516239     0.176874
 -0.290837    0.416539    0.299599    0.267149     0.224628
 -0.528765    0.686753    0.689029    0.530688     0.478941
 -0.00373652  0.64302     0.544063    0.543674     1.02829
  ⋮                                               
 -0.160819    0.57642     0.506932    0.453938     0.198784
 -0.258833    0.599308    0.513425    0.458414     0.272149
 -0.40437     0.538775   

In [52]:
Matrix(F.Q')[1:Ne,:]*ψ_diag'

100×4 Array{Float64,2}:
  0.259283   -1.26526     -1.14093    -1.10693
  2.43358    -4.98023     -4.15672    -3.03175
 -0.113712    1.00018      0.839181    0.0215528
  0.238057    0.185024     0.180396    0.246032
 -0.527554   -1.92636     -1.58001    -1.23033
 -0.0281731  -0.576664    -0.454201   -0.117531
  0.0324086   0.258376     0.112765   -0.40009
  0.999858   -0.360184    -0.288807    0.0484478
  0.189334   -0.356912    -0.284213   -0.163268
  0.168473    0.0464709    0.0292691  -0.050503
  0.324687    0.0042863   -0.0607825  -0.230189
  0.0883382   0.0648405    0.0460566   0.0261449
  0.0130686  -0.0570624   -0.0624704  -0.312733
  ⋮                                   
  0.236156    0.155029     0.0823092  -0.185805
  0.060593   -0.00593668  -0.0453959  -0.131984
  0.132133    0.524442     0.388536    0.193398
  0.0585351  -0.425783    -0.352708   -0.397807
  0.109684   -0.0136371   -0.0583209  -0.322824
  0.190798    0.189964     0.0581992  -0.237499
  0.185859    0.0863229   

In [44]:
F = qr(ψ_off')

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.17235     -0.0123195   0.0195756   …   0.0552232   -0.342795
  0.0477271   -0.090474    0.140895       -0.131694     0.00507162
 -0.0343564   -0.124909   -0.0934567       0.143054     0.15335
 -0.00692869  -0.124303   -0.00655872      0.0112888   -0.0174998
  0.0110816   -0.116911    0.05368        -0.08452     -0.0441607
  0.039191    -0.0974732   0.127091    …   0.129849    -0.180872
 -0.00775866  -0.124519   -0.00939141     -0.131624     0.0209253
  0.181403    -0.0221591  -0.0599955       0.0374523    0.0550513
 -0.133454    -0.0445787  -0.0776178       0.0188524    0.047145
  0.0575559   -0.0822022   0.150916       -0.0249855   -0.0082464
 -0.00117364  -0.122488    0.0130785   …   0.00616488  -0.0233737
  0.13222     -0.0328142   0.0549326       0.0011328    0.00471768
 -0.0266938   -0.126127   -0.0713438      -0.0067487   -0.0146783
  ⋮                  

In [140]:
using BenchmarkTools
N = 200
nx = 597
λ = 0.1

ψ = rand(nx,N)
ψ_aug = zeros(N+nx, nx)

ψ_aug[1:N,:] .= ψ'
ψ_aug[N+1:N+nx,:] .= Matrix(sqrt(λ)*I, nx, nx)

@btime F = qr(ψ_aug)

Q1 = zeros(N, nx)
@btime Q1 .= view(F.Q*Matrix(I, N+nx, N+nx), 1:N, 1:nx)
@btime Q1 .= (F.Q*Matrix(I,N+nx, N+nx))[1:N, 1:nx]
@btime Q1 .= Matrix{Float64}(F.Q)[1:N, 1:nx];


ψbis  = rand(4, N) 

A = ψbis - (ψbis*Q1)*Q1';

  13.222 ms (7 allocations: 3.96 MiB)
  11.901 ms (15 allocations: 9.08 MiB)
  11.224 ms (16 allocations: 9.99 MiB)
  9.262 ms (14 allocations: 10.89 MiB)


In [141]:
@btime f($ψbis, $(F.Q), $N, $nx);

  951.466 μs (18 allocations: 187.88 KiB)


In [159]:
@testset "Verify updateqrfactUnblocked! for a vector" begin

    ψ = randn(100, 20)
    ϕ = randn(100)

    F = qrfactUnblocked(ψ)
    Fold = deepcopy(F)
    G = qrfactUnblocked(hcat(ψ, ϕ))
    F = updateqrfactUnblocked!(F, ϕ)
    @test norm(F.τ-G.τ)<1e-14
    @test norm(F.factors - G.factors)<1e-14
end

@testset "Verify updateqrfactUnblocked! for a matrix" begin

    ψ = randn(100, 20)
    ϕ = randn(100, 5)

    F = qrfactUnblocked(ψ)
    Fold = deepcopy(F)
    G = qrfactUnblocked(hcat(ψ, ϕ))
    F = updateqrfactUnblocked!(F, ϕ)
    @test norm(F.τ-G.τ)<1e-14
    @test norm(F.factors - G.factors)<1e-14
end

Test Summary:                              | Pass  Total
Verify updateqrfactUnblocked! for a vector |    2      2
Test Summary:                              | Pass  Total
Verify updateqrfactUnblocked! for a matrix |    2      2


Test.DefaultTestSet("Verify updateqrfactUnblocked! for a matrix", Any[], 2, false)